In [2]:
using Makie, TopOpt, LinearAlgebra, StatsFuns
using TopOpt.TopOptProblems.Visualization: visualize

## Parameter setup

In [3]:
E = 1.0 # Young’s modulus
v = 0.3 # Poisson’s ratio
f = 1.0 # downward force
rmin = 4.0 # filter radius
xmin = 0.0001 # minimum density
problem_size = (30, 10, 10)
V = 0.5 # maximum volume fraction
p = 4.0 # penalty

x0 = fill(V, prod(problem_size)); # initial design

## Define a new problem

In [4]:
using Ferrite
using TopOpt
using TopOpt.TopOptProblems: RectilinearGrid, Metadata
using TopOpt.TopOptProblems: left, right, bottom, middley, middlez,
    nnodespercell, nfacespercell, find_black_and_white, find_varind
using TopOpt.Utilities: @params

@params struct NewPointLoadCantilever{dim, T, N, M} <: StiffnessTopOptProblem{dim, T}
    rect_grid::RectilinearGrid{dim, T, N, M}
    E::T
    ν::T
    ch::ConstraintHandler{<:DofHandler{dim, <:Cell{dim,N,M}, T}, T}
    load_dict::Dict{Int, Vector{T}}
    black::AbstractVector
    white::AbstractVector
    varind::AbstractVector{Int}
    metadata::Metadata
end

function NewPointLoadCantilever(::Type{Val{CellType}}, nels::NTuple{dim,Int}, sizes::NTuple{dim}, 
    E = 1.0, ν = 0.3, force = 1.0) where {dim, CellType}
    iseven(nels[2]) && (length(nels) < 3 || iseven(nels[3])) || throw("Grid does not have an even number of elements along the y and/or z axes.")

    _T = promote_type(eltype(sizes), typeof(E), typeof(ν), typeof(force))
    if _T <: Integer
        T = Float64
    else
        T = _T
    end
    if CellType === :Linear || dim === 3
        rect_grid = RectilinearGrid(Val{:Linear}, nels, T.(sizes))
    else
        rect_grid = RectilinearGrid(Val{:Quadratic}, nels, T.(sizes))
    end

    if haskey(rect_grid.grid.facesets, "fixed_all") 
        pop!(rect_grid.grid.facesets, "fixed_all")
    end
    addnodeset!(rect_grid.grid, "fixed_all", x -> left(rect_grid, x));
    
    if haskey(rect_grid.grid.nodesets, "down_force") 
        pop!(rect_grid.grid.nodesets, "down_force")
    end
    if dim == 3
        addnodeset!(rect_grid.grid, "down_force", x -> right(rect_grid, x) && 
            bottom(rect_grid, x));
            #  && middlez(rect_grid, x));
    else
        addnodeset!(rect_grid.grid, "down_force", x -> right(rect_grid, x) && 
            right(rect_grid, x) && middley(rect_grid, x));
    end

    # Create displacement field u
    dh = DofHandler(rect_grid.grid)
    if CellType === :Linear || dim === 3
        push!(dh, :u, dim) # Add a displacement field
    else
        ip = Lagrange{2, RefCube, 2}()
        push!(dh, :u, dim, ip) # Add a displacement field        
    end
    close!(dh)
    
    ch = ConstraintHandler(dh)

    dbc = Dirichlet(:u, getnodeset(rect_grid.grid, "fixed_all"), (x,t) -> zeros(T, dim), collect(1:dim))
    add!(ch, dbc)
    close!(ch)
    t = T(0)
    Ferrite.update!(ch, t)

    metadata = Metadata(dh)
    load_dict = Dict{Int, Vector{T}}()
    for fnode in getnodeset(rect_grid.grid, "down_force")
    	load_dict[fnode] = [0, -force, 0]
    end

    N = nnodespercell(rect_grid)
    M = nfacespercell(rect_grid)

    black, white = find_black_and_white(dh)
    varind = find_varind(black, white)
    
    return NewPointLoadCantilever(rect_grid, E, ν, ch, load_dict, black, white, varind, metadata)
end

# used in FEA to determine default quad order
# we don't assume the problem struct has `rect_grid` to define its grid
TopOptProblems.nnodespercell(p::NewPointLoadCantilever) = nnodespercell(p.rect_grid)

# ! important, used for specification!
function TopOptProblems.getcloaddict(p::NewPointLoadCantilever{dim, T}) where {dim, T}
    # f = T[0, -p.force, 0]
    # fnode = Tuple(getnodeset(p.rect_grid.grid, "down_force"))[1]
    # return Dict{Int, Vector{T}}(fnode => f)
    return p.load_dict
end

In [5]:
problem = NewPointLoadCantilever(Val{:Linear}, problem_size, (1.0, 1.0, 1.0), E, v, f);

## FEA solvers

In [6]:
solver = FEASolver(Displacement, Direct, problem, xmin = xmin);

In [7]:
solver()
u0 = solver.u

11253-element Vector{Float64}:
    0.0
    0.0
    0.0
   -2.2099446376081686
   -0.9999034775224457
   -1.4202725587371576
   -1.5759577627647248
   -0.42196268331009407
   -1.140815179503329
    0.0
    0.0
    0.0
    0.0
    ⋮
   28.54202289729392
 -106.73217916326222
   -0.19481464550894279
   28.60298050880803
 -112.64039125867208
   -0.1259155145335127
   28.61626113360229
 -118.4816362695016
   -0.08202963419333631
   28.614817608618498
 -124.26831756075559
   -0.05206099717716567

In [8]:
# visualize the deformation!
fig = visualize(problem, u0)
Makie.display(fig)

GLMakie.Screen(...)

In [ ]:
# cg_assembly_solver = FEASolver(Displacement, CG, Assembly, problem, xmin = xmin)
# cg_matrix_free_solver = FEASolver(Displacement, CG, MatrixFree, problem, xmin = xmin)

## Construct objective and constraints (function definitions)

In [ ]:
cheqfilter = DensityFilter(solver, rmin = rmin)
stress = TopOpt.MicroVonMisesStress(solver)
comp = TopOpt.Compliance(problem, solver)

# minimize compliance
function obj(x)
    return comp(cheqfilter(x))
end

# volume bound
function constr(x)
    return sum(cheqfilter(x)) / length(x) - V
end;

## Run optimization

In [ ]:
m = Model(obj)
addvar!(m, zeros(length(x0)), ones(length(x0)))
Nonconvex.add_ineq_constraint!(m, constr)

## enable printout
# Nonconvex.show_residuals[] = true

options = MMAOptions(
    maxiter=300, tol = Tolerance(kkt = 1e-3, x=1e-3, f = 1e-3),
)
TopOpt.setpenalty!(solver, p)

@time r = Nonconvex.optimize(
    m, MMA87(dualoptimizer = ConjugateGradient()),
    x0, options = options,
);

In [ ]:
# show the optimized results!
@show obj(r.minimizer)
@show constr(r.minimizer)
@show maximum(stress(cheqfilter(r.minimizer)))

topology = cheqfilter(r.minimizer);
fig = visualize(problem; topology = topology)
Makie.display(fig)

## Call FEA solver on a specific topology

In [ ]:
x1 = rand(length(solver.vars))

In [ ]:
solver.vars = x1
solver()
u1 = solver.u

In [ ]:
# visualize the deformation!
fig = visualize(problem, u1; topology=x1, default_exagg_scale=0.001)
Makie.display(fig)